In [1]:
import numpy as np

import pandas as pd
import os
import glob
import scipy
import gzip
import pickle
import multiprocessing

import seaborn as sns
import matplotlib.pyplot as plt 

import raycastingfunctions as rc
import detectionfunctions as detection
import importlib

import time

In [2]:
# import
trials=['10-fish/0066/','/10-fish/0105/','/10-fish/0126/','30-fish/0084/','/30-fish/0115/','/30-fish/0120/','70-fish/0103/','70-fish/0107/','70-fish/0124/','150-fish/']
framerate=30
dt=1/framerate

datadir = '../data/'
detectiondir = '../detectionresults/'

# here, blind_angle is used to set the region of binocular vision.  using symmetry
blind_angle = 25/360*np.pi*2  # S LeBlanc used 25 degrees, citing ref:  Diana Pita, Bret A. Moore, Luke P. Tyrrell, Esteban Fernández-Juricic, and Marı́a Ángeles Esteban. Vision in two cyprinid fish: implications for collectivebehavior. PeerJ, 3:e1113, 2015. doi: 10.7717/peerj.1113. URL https://doi.org/10.7717/peerj.1113.
binocularangle = blind_angle/2

# Run external detection simulations

## Probabilistic blockage and full blockage

In [9]:
numprocesses = 10
processing_skipvalue = 5

In [ ]:
blockingprobs=[0.75,0.5]

for tnum in range(len(trials)):
    trial=trials[tnum]
    print('trial: ',trial)

    print('\n\n',basedir+trial)

    outname=basedir+trial+'basicdata-v3.pklz'
    [positions,orientations,tailcoords,lefteye,righteye,groupcentroid,groupheading,groupheadingxy,groupstates,pixelwidth,pixelheight,rotcoords] = pickle.load(gzip.open(outname,'rb'))
    frontbackdist=rotcoords[:,:,0]
    sidesidedist=rotcoords[:,:,1] 

    numsteps,numfish, _ = positions.shape

    ## Run for different point positions

    pointdist = 1200
    numsegperfish = 4
    segids = np.hstack(np.tile(np.arange(numfish),(numsegperfish,1)).T)
    numpoints = 200

    allseen = np.zeros((numsteps,numfish,numpoints))
    allseen_high = np.zeros((numsteps,numfish,numpoints))
    allseen_low = np.zeros((numsteps,numfish,numpoints))
    allrelangle = np.zeros((numsteps,numfish,numpoints))
    for step in range(0,numsteps,processing_skipvalue):
        if np.mod(step,500)==0:
            print(step,numsteps)
        allpoints = detection.getpointpositions_many(groupcentroid[step],groupheadingxy[step],dist=pointdist,numpoints=numpoints)        
#         segstep = detection.getbonesegsinglestep(positions[step],midcoords[step],skip=numskip)  # line model - old
        fsegs = detection.fishmodel(positions[step],lefteye[step],righteye[step],tailcoords[step])
        segstep = np.reshape(fsegs,(-1,2,2))        
        
        pool = multiprocessing.Pool(processes=numprocesses)
        def processfish(i):
            # don't include the right-side polygon for the right eye, or the left side polygon for the left eye
            mask = np.tile(True,len(segstep))
            mask_le = mask.copy()
            mask_re = mask.copy()
            mask_le[i*4+2:i*4+4] = False
            mask_re[i*4:i*4+2] = False

            # generate random numbers for probabilistic intersection test
            # random vales are perfish
            randperfish = np.random.rand(numfish)
            randpersegments = np.hstack(np.tile(randperfish,(numsegperfish,1)).T)
            randvalues = np.tile(randpersegments,(numpoints,1)).T
            randvalues[segids==i]=0  # the fish's own body always blocks itself

            # calculate relate point orientation for blind spot
            pos = positions[step,i]
            orient = orientations[step,i]
            dpos = allpoints - pos
            dangle = np.arctan2(dpos[:,1],dpos[:,0])
            relangle = detection.fixanglerange(dangle-orient)

            le, re = lefteye[step,i], righteye[step,i] 
            da = allpoints - le
            lpointangles = np.arctan2(da[:,1],da[:,0])
            lseen = detection.getoneray_intersecttest(le,segstep[mask_le],lpointangles)
            lseen_high = detection.getoneray_intersecttest_probabilistic(le,segstep[mask_le],lpointangles,randvalues[mask_le],blockingprobs[0])
            lseen_low = detection.getoneray_intersecttest_probabilistic(le,segstep[mask_le],lpointangles,randvalues[mask_le],blockingprobs[1])            

            da = allpoints - re
            rpointangles = np.arctan2(da[:,1],da[:,0])
            rseen = detection.getoneray_intersecttest(re,segstep[mask_re],rpointangles)
            rseen_high = detection.getoneray_intersecttest_probabilistic(re,segstep[mask_re],rpointangles,randvalues[mask_re],blockingprobs[0])    
            rseen_low = detection.getoneray_intersecttest_probabilistic(re,segstep[mask_re],rpointangles,randvalues[mask_re],blockingprobs[1])    

            # set the region of overlap
            lrelangle = detection.fixanglerange(lpointangles-orient)
            lsel = lrelangle<-binocularangle
            lseen[lsel]=0
            lseen_high[lsel]=0          
            lseen_low[lsel]=0

            rrelangle = detection.fixanglerange(rpointangles-orient)
            rsel = rrelangle>binocularangle
            rseen[rsel]=0
            rseen_high[rsel]=0
            rseen_low[rsel]=0

            return lseen, rseen, lseen_high, rseen_high, lseen_low, rseen_low, relangle

        results = np.array(pool.map(processfish,range(numfish)))
        pool.close()

        allseen[step] = np.logical_or(results[:,0],results[:,1])  # np.logical_or(lseen,rseen)
        allseen_high[step] = np.logical_or(results[:,2],results[:,3]) #np.logical_or(lseen_high,rseen_high)
        allseen_low[step] = np.logical_or(results[:,4],results[:,5])  #np.logical_or(lseen_low,rseen_low)      
        allrelangle[step] = results[:,6]

    outdir = detectiondir + trial
    os.mkdir(outdir)
    outname = outdir + 'detection-probabilistic-perfish.pklz'
    pickle.dump([pointdist,allseen,allseen_high,allseen_low,allrelangle,processing_skipvalue],gzip.open(outname,"wb"),protocol=4)